In [1]:
cd ..

/home/jovyan


In [2]:
import matplotlib.pyplot as plt
%matplotlib inline
import lib.lm as lm
from lib.preprocessing import BoxCoxTransformer
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tqdm
from sklearn.datasets import load_boston
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.exceptions import ConvergenceWarning

In [3]:
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=ConvergenceWarning)

In [4]:
boston = load_boston()

In [5]:
X = pd.DataFrame(boston.data, columns=boston.feature_names)

In [6]:
y = boston.target

In [7]:
X.sample(4)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
247,0.19657,22.0,5.86,0.0,0.431,6.226,79.2,8.0555,7.0,330.0,19.1,376.14,10.15
309,0.34940,0.0,9.90,0.0,0.544,5.972,76.7,3.1025,4.0,304.0,18.4,396.24,9.97
405,67.92080,0.0,18.10,0.0,0.693,5.683,100.0,1.4254,24.0,666.0,20.2,384.97,22.98
391,5.29305,0.0,18.10,0.0,0.700,6.051,82.5,2.1678,24.0,666.0,20.2,378.38,18.76


## Standardization

Regularized linear models require standardization and possibly skew normalization to function properly.

Note from both the statistical description that features differ significantly in terms of mean and standard deviation. This will impact regularization performance.

In [8]:
samp_stats = X.describe().T
samp_stats

,count,mean,std,min,25%,50%,75%,max
CRIM,506.0,3.593761,8.596783,0.00632,0.082045,0.25651,3.647423,88.9762
ZN,506.0,11.363636,23.322453,0.00000,0.000000,0.00000,12.500000,100.0000
INDUS,506.0,11.136779,6.860353,0.46000,5.190000,9.69000,18.100000,27.7400
CHAS,506.0,0.069170,0.253994,0.00000,0.000000,0.00000,0.000000,1.0000
NOX,506.0,0.554695,0.115878,0.38500,0.449000,0.53800,0.624000,0.8710
RM,506.0,6.284634,0.702617,3.56100,5.885500,6.20850,6.623500,8.7800
AGE,506.0,68.574901,28.148861,2.90000,45.025000,77.50000,94.075000,100.0000
DIS,506.0,3.795043,2.105710,1.12960,2.100175,3.20745,5.188425,12.1265
RAD,506.0,9.549407,8.707259,1.00000,4.000000,5.00000,24.000000,24.0000
TAX,506.0,408.237154,168.537116,187.00000,279.000000,330.00000,666.000000,711.0000


## Standardized Model


In [9]:
results = pd.DataFrame()

In [10]:
results, _, _ = \
    lm.simple_alpha_grid_search(make_pipeline(StandardScaler(), LinearRegression()), X, y, results, n=1000)

100%|██████████| 1000/1000 [00:04<00:00, 243.42it/s]


In [11]:
results

,alpha,data_preprocessing,model,model_name,test_score,train_score
0,NaN,[standardscaler],"Pipeline(steps=[('standardscaler', StandardSca...",linearregression,0.711009,0.743956


In [12]:
results, _, _ = \
    lm.simple_alpha_grid_search(make_pipeline(StandardScaler(), Lasso()), X, y, results, n=1000)

100%|██████████| 1000/1000 [00:03<00:00, 293.61it/s]


In [13]:
results, _, _ = \
    lm.simple_alpha_grid_search(make_pipeline(StandardScaler(), Ridge()), X, y, results, n=1000)

100%|██████████| 1000/1000 [00:03<00:00, 318.62it/s]


In [14]:
results

,alpha,data_preprocessing,model,model_name,test_score,train_score
0,NaN,[standardscaler],"Pipeline(steps=[('standardscaler', StandardSca...",linearregression,0.711009,0.743956
0,0.01,[standardscaler],"Pipeline(steps=[('standardscaler', StandardSca...",lasso,0.711215,0.743880
0,10.00,[standardscaler],"Pipeline(steps=[('standardscaler', StandardSca...",ridge,0.711298,0.742562
